# Rapid Model Analysis Based on Attribute Presence for Product Master Data Management

Author: **Marcin Sikorski**<br>
Date: April, 2022

Our PIM (Product Information Management) system has loads of models (approximately 3000) and we have suspicions that they may repeat themselves. Our goal is to verify if any models are the same and reduce their number if possible. We can do this task by manually checking each row in each model, which of course is time-consuming and can take the MDM team a few weeks or months to complete. We can perform this task in Python by comparing each model with eachother for verification.


Business assumptions for this case study:
1. In this particular system, a model is an entity which contains a set of attributes (characteristics) that describe given model. The characteristics then contain master data.
2. Each new product introduced into the system is assigned to the model based on its physical features (for example: dimensions, color, engine power, tightening torque, surface, product brand, etc.).
3. In the context of attributes, each model should be unique.

In [1]:
import pandas as pd

pd.read_excel('tree_charac.xlsx', header=0).head(10)

,Model ID,Model name (EN),Characteristic ID,Characteristic name (EN)
0,MOD_200001,Hinged sectional ladder,ATT_01022,Description
1,MOD_200001,Hinged sectional ladder,ATT_00055,Depth (in cm)
2,MOD_200001,Hinged sectional ladder,ATT_01060,Maximum load per bar (in kg)
3,MOD_200001,Hinged sectional ladder,ATT_00052,Commercial reference
4,MOD_200001,Hinged sectional ladder,ATT_00124,Net weight (in kg)
5,MOD_200001,Hinged sectional ladder,ATT_01181,More info about the product
6,MOD_200001,Hinged sectional ladder,ATT_01181,More info about the product
7,MOD_200001,Hinged sectional ladder,ATT_01181,More info about the product
8,MOD_200001,Hinged sectional ladder,ATT_01181,More info about the product
9,MOD_200001,Hinged sectional ladder,ATT_01181,More info about the product


In [2]:
# creating a sample dataframe
def prepare_dataframe():
    sample = pd.read_excel('tree_charac.xlsx', header=0)
    sample.drop(['Model name (EN)', 'Characteristic name (EN)'], axis=1, inplace=True)
    sample = sample.drop_duplicates(subset=['Model ID', 'Characteristic ID'], keep='first')
    return sample

sample = prepare_dataframe()
sample.head()

,Model ID,Characteristic ID
0,MOD_200001,ATT_01022
1,MOD_200001,ATT_00055
2,MOD_200001,ATT_01060
3,MOD_200001,ATT_00052
4,MOD_200001,ATT_00124


In [3]:
# sample description
def print_values():
    print('Rows:', sample.shape[0])
    print('Unique models:', sample['Model ID'].nunique())
    print('Unique characteristics:', sample['Characteristic ID'].nunique())

print_values()

Rows: 1519
Unique models: 15
Unique characteristics: 364


In [4]:
# number of variations with repitition
sample['Model ID'].nunique() * sample['Model ID'].nunique()

225

One way for representing and verifying attribute presence per model is by a frequency matrix. Each row represents a model and each column demonstrates a unique characteristic from our sample. The cells are defined by this specific coordinate system. 0 means non-existance, and 1 means the feature is in use. Any attribute must exist at least in one model. To reduce the size of the table we can drop columns that have all rows equal 1.

Although this type of graph makes verifying much easier it still does not solve completely our issue. The user still must check data row by row which is time consuming and exhausting.

In [5]:
# frequency matrix
pd.crosstab(sample['Model ID'], sample['Characteristic ID'])

Characteristic ID,ATT_00034,ATT_00040,ATT_00052,ATT_00053,ATT_00054,ATT_00055,ATT_00056,ATT_00084,ATT_00124,ATT_00206,...,ATT_25930,ATT_25931,ATT_25933,ATT_26151,ATT_26156,ATT_26187,ATT_26218,ATT_26219,ATT_26248,ATT_26350
Model ID,,,,,,,,,,,,,,,,,,,,,
MOD_200001,0,1,1,1,0,1,0,1,1,0,...,0,0,0,1,1,1,1,1,0,0
MOD_200002,0,0,1,0,0,0,0,1,1,0,...,0,0,0,1,1,1,1,1,0,0
MOD_200003,0,0,1,0,0,0,0,1,1,0,...,0,0,0,1,1,1,1,1,0,0
MOD_200004,0,0,1,0,0,0,0,1,1,0,...,0,0,0,1,1,1,1,1,0,0
MOD_200005,0,0,1,1,1,1,0,1,1,0,...,0,0,0,1,1,1,1,1,0,1
MOD_200006,0,0,1,1,1,1,0,1,1,1,...,0,0,0,1,1,1,1,1,0,0
MOD_200007,0,0,1,0,0,0,0,1,1,0,...,0,0,0,1,1,1,1,1,0,0
MOD_200008,0,0,1,0,0,0,0,1,1,0,...,0,0,0,1,1,1,1,1,0,0
MOD_200009,0,0,1,1,0,0,0,1,1,0,...,1,1,1,1,1,1,1,1,1,0


In [6]:
# list of model id's
mod_list = sample['Model ID'].unique().tolist()

# creating list of dataframes (list of unique models)
# -- sort attributes ascending and reset index for each item in list
def list_dataframes():
    df_list = []
    for i in mod_list:
        df_list.append(sample.loc[sample['Model ID'] == i].sort_values(by=['Characteristic ID'], \
                                                                   ascending=True).reset_index(drop=True))
    return df_list
    
df_list = list_dataframes()

# numer of elements (should be 15)
len(df_list)

15

In [7]:
# verification - displaying any element from list
df_list[9]

,Model ID,Characteristic ID
0,MOD_200010,ATT_00052
1,MOD_200010,ATT_00053
2,MOD_200010,ATT_00054
3,MOD_200010,ATT_00055
4,MOD_200010,ATT_00084
...,...,...
87,MOD_200010,ATT_26151
88,MOD_200010,ATT_26156
89,MOD_200010,ATT_26187
90,MOD_200010,ATT_26218


We defined a list which stores dataframes as unique items. Each item (or dataframe) represents a model with its characteristics. By performing a cross join (iterating the list), we can compare each model for duplicates. 'False' indicates no repetition, and of course 'true' means the models are the same.

In [8]:
# final function - with model id's
def check_models():
    for i in range(0, len(mod_list)): 
        for j in range(0, len(mod_list)):
            if i == j:
                pass
            else:
                print(mod_list[i], '|', mod_list[j], '|', df_list[i].iloc[:, 1].equals(df_list[j].iloc[:, 1]))
                
check_models()

MOD_200001 | MOD_200002 | False
MOD_200001 | MOD_200003 | False
MOD_200001 | MOD_200004 | False
MOD_200001 | MOD_200005 | False
MOD_200001 | MOD_200006 | False
MOD_200001 | MOD_200007 | False
MOD_200001 | MOD_200008 | False
MOD_200001 | MOD_200009 | False
MOD_200001 | MOD_200010 | False
MOD_200001 | MOD_200011 | False
MOD_200001 | MOD_200012 | False
MOD_200001 | MOD_200013 | False
MOD_200001 | MOD_200014 | False
MOD_200001 | MOD_200015 | False
MOD_200002 | MOD_200001 | False
MOD_200002 | MOD_200003 | False
MOD_200002 | MOD_200004 | False
MOD_200002 | MOD_200005 | False
MOD_200002 | MOD_200006 | False
MOD_200002 | MOD_200007 | False
MOD_200002 | MOD_200008 | False
MOD_200002 | MOD_200009 | False
MOD_200002 | MOD_200010 | False
MOD_200002 | MOD_200011 | False
MOD_200002 | MOD_200012 | False
MOD_200002 | MOD_200013 | False
MOD_200002 | MOD_200014 | False
MOD_200002 | MOD_200015 | False
MOD_200003 | MOD_200001 | False
MOD_200003 | MOD_200002 | False
MOD_200003 | MOD_200004 | False
MOD_2000

In [9]:
# much readable output - with list index
def index_models():
    for i in range(0, len(mod_list)): 
        for j in range(0, len(mod_list)):
            if i == j:
                pass
            else:
                print(i, j, df_list[i].iloc[:, 1].equals(df_list[j].iloc[:, 1]))
                
index_models()

0 1 False
0 2 False
0 3 False
0 4 False
0 5 False
0 6 False
0 7 False
0 8 False
0 9 False
0 10 False
0 11 False
0 12 False
0 13 False
0 14 False
1 0 False
1 2 False
1 3 False
1 4 False
1 5 False
1 6 False
1 7 False
1 8 False
1 9 False
1 10 False
1 11 False
1 12 False
1 13 False
1 14 False
2 0 False
2 1 False
2 3 False
2 4 False
2 5 False
2 6 False
2 7 False
2 8 False
2 9 False
2 10 False
2 11 False
2 12 False
2 13 False
2 14 False
3 0 False
3 1 False
3 2 False
3 4 False
3 5 False
3 6 False
3 7 False
3 8 False
3 9 False
3 10 False
3 11 False
3 12 False
3 13 False
3 14 False
4 0 False
4 1 False
4 2 False
4 3 False
4 5 False
4 6 False
4 7 False
4 8 False
4 9 False
4 10 False
4 11 False
4 12 False
4 13 False
4 14 False
5 0 False
5 1 False
5 2 False
5 3 False
5 4 False
5 6 False
5 7 False
5 8 False
5 9 False
5 10 False
5 11 False
5 12 False
5 13 False
5 14 False
6 0 False
6 1 False
6 2 False
6 3 False
6 4 False
6 5 False
6 7 False
6 8 False
6 9 False
6 10 False
6 11 False
6 12 False
6 13 Fa

### Conclusion

In this case study we wanted to accelerate and automate as possible model analysis for PMDM. The verification was based on the analysis of the presence of characteristics in these models with a cross join. We successfully performed this task on a sample dataset with 15 models and none of them are repeated. The above operations not only significantly accelerated the process but also allow for the elimination of user errors when reviewing each model.